In [47]:
import pandas as pd
df = pd.read_csv('data/cleaned/merged_house_finance_data_all_years.csv')

In [48]:
df[df['pvi'].isna()].state.value_counts()

Series([], Name: count, dtype: int64)

In [49]:
df = df[~df['party'].isna()]
df

,id,year,state,state_po,district,runoff,special,candidate_name,party,candidate_votes,total_votes,FEC_candidate_id,CAND_ICI,TTL_DISB,NHE,pvi
0,2010AK0,2010,ALASKA,AK,0,NaN,False,DON YOUNG,REPUBLICAN,175384,254335,H6AK00045,I,887310.33,6.8,13.0
1,2010AK0,2010,ALASKA,AK,0,NaN,False,HARRY T CRAWFORD JR,DEMOCRAT,77606,254335,H0AK00089,C,235571.43,6.8,13.0
2,2010AL1,2010,ALABAMA,AL,1,NaN,False,DAVID WALTER,CONSTITUTION,26357,156281,H0AL01048,C,20045.00,6.8,14.0
3,2010AL1,2010,ALABAMA,AL,1,NaN,False,JO BONNER,REPUBLICAN,129063,156281,H2AL01077,I,1101701.00,6.8,14.0
4,2010AL2,2010,ALABAMA,AL,2,NaN,False,BOBBY BRIGHT,DEMOCRAT,106865,219028,H8AL02130,I,1435626.44,6.8,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7689,2020WV3,2020,WEST VIRGINIA,WV,3,False,False,HILARY TURNER,DEMOCRAT,64927,226668,H0WV03151,C,98768.58,-3.1,23.0
7690,2020WY0,2020,WYOMING,WY,0,False,False,JEFF HAGGIT,CONSTITUTION,7905,278503,NaN,NaN,NaN,-3.1,25.0
7691,2020WY0,2020,WYOMING,WY,0,False,False,LIZ CHENEY,REPUBLICAN,185732,278503,H6WY00159,I,3066534.91,-3.1,25.0
7692,2020WY0,2020,WYOMING,WY,0,False,False,LYNNETTE GREY BULL,DEMOCRAT,66576,278503,H0WY01036,C,132285.65,-3.1,25.0


In [50]:
df['percent_of_vote'] = (df['candidate_votes'] / df['total_votes']).round(4)

In [51]:
# Calculate the difference between the highest and second highest percent_of_vote for each 'id'
df_sorted = df.sort_values(['id', 'percent_of_vote'], ascending=[True, False])
df_top2 = df_sorted.groupby('id').head(2)
df_diff = (-1 * df_top2.groupby('id')['percent_of_vote'].diff()).fillna(0).round(4)
df_margin = df_diff.groupby(df_top2['id']).last().reset_index()
df_margin.columns = ['id', 'vote_margin']

# Merge the vote margin back to the original dataframe
df_with_margin = df.merge(df_margin, on='id', how='left')

In [52]:
df_with_margin['adjusted_vote_margin'] = df_with_margin.apply(lambda row: 
    round(row['vote_margin'] - row['pvi'] - row['NHE'], 2) if row['party'] == 'DEMOCRAT' else
    round(row['vote_margin'] + row['pvi'] + row['NHE'], 2) if row['party'] == 'REPUBLICAN' else
    float('nan'), axis=1)

In [53]:
df_with_margin.columns

Index(['id', 'year', 'state', 'state_po', 'district', 'runoff', 'special',
       'candidate_name', 'party', 'candidate_votes', 'total_votes',
       'FEC_candidate_id', 'CAND_ICI', 'TTL_DISB', 'NHE', 'pvi',
       'percent_of_vote', 'vote_margin', 'adjusted_vote_margin'],
      dtype='object')

In [56]:
df_with_margin = df_with_margin[['id', 'year', 'state', 'state_po', 'district', 'runoff', 'special',
       'candidate_name', 'party', 'CAND_ICI', 'TTL_DISB', 'adjusted_vote_margin', 'candidate_votes', 'total_votes', 'NHE', 'pvi',
       'percent_of_vote', 'vote_margin',
       'FEC_candidate_id']]

df_with_margin['party'] = df_with_margin['party'].apply(lambda x: 'Democrat' if x.lower() == 'democrat' else 'Republican' if x.lower() == 'republican' else 'Other')

In [57]:
df_with_margin.to_csv('data/cleaned/all_final.csv', index=False)